[Reference](https://towardsdatascience.com/road-transportation-optimization-with-python-part-1-visualisation-costing-698eadcdce0b)

In [ ]:
# Create Transport Plan
def transport_plan(data, dict_trucks, capacity_dict):
	# List of Stores per Truck for each DAY
	df_plan = pd.DataFrame(data.groupby(['Date', 'TruckID'])['Code'].apply(list))
	df_plan.columns = ['List_Code']
	# List of Box Quantity
	df_plan['List_BOX'] = data.groupby(['Date', 'TruckID'])['BOX'].apply(list)
	# Mean of FTL
	df_plan['FTL']  = data.groupby(['Date', 'TruckID'])['FTL'].mean()
	df_plan['Capacity(T)'] = df_plan['FTL'].map(capacity_dict)
	df_plan['List_Loading'] = data.groupby(['Date', 'TruckID'])['Loading(T)'].apply(list)
	df_plan['Count'] = df_plan['List_Loading'].apply(lambda t: len(t))
	df_plan['Total_tons(T)'] = data.groupby(['Date', 'TruckID'])['Loading(T)'].sum()

	# Distribute: one shipment per col
	# Stores
	d = df_plan['List_Code'].apply(pd.Series)
	for col in d:
	    df_plan["Store%d" % (col+1)] = d[col]
	# Boxes number    
	d = df_plan['List_BOX'].apply(pd.Series)
	for col in d:
	    df_plan["Box%d" % (col+1)] = d[col]
	# Shipments Tonnage
	d = df_plan['List_Loading'].apply(pd.Series)
	for col in d:
	    df_plan["Tons%d" % (col+1)] = d[col]

	# Fill NaN + Drop useless columns
	df_plan.fillna(0, inplace = True)
	if 1 == 0:
		df_plan.drop(['List_Code'], axis = 1, inplace = True)
		df_plan.drop(['List_BOX'], axis = 1, inplace = True)
		df_plan.drop(['List_Loading'], axis = 1, inplace = True)

	return df_plan

In [ ]:
## Pricing Functions
def f_maxcity(list_cities, list_price):
	return list_cities[list_price.index(max(list_price))] # Index of Maximum Price
def inner_stops(list_cities, max_city):
	return list_cities.count(max_city) - 1
def outer_stops(list_cities, max_city):
	return len(list_cities) - (list_cities.count(max_city))
def total_price(max_price, inner_stops, outer_stops, inner_price, outer_price):
	return max_price + inner_stops * inner_price + outer_stops * outer_price

# Calculate Price
def plan_price(df_strinfo, df_plan, inner_price, outer_price):

	# Dictionnary Ville
	dict_ville = dict(zip(df_strinfo.Code.values, df_strinfo.City.values))

	# Price per Truck Size： 3.5T, 5T, 8T
	dict_35, dict_5, dict_8 = [dict(zip(df_strinfo.City.values, df_strinfo[col].values)) for col in ['3.5T', '5T', '8T']]

	# Mapping Cities
	f_ville = lambda t: [dict_ville[i] for i in t] # literal_eval(t)

	# Mapping Price
	f_35 = lambda t: [dict_35[i] for i in t]
	f_5 = lambda t: [dict_5[i] for i in t]
	f_8 = lambda t: [dict_8[i] for i in t]

	# Mapping Price
	df_plan['List_City'] = df_plan['List_Code'].map(f_ville)
	df_plan['List_Price35'] = df_plan['List_City'].map(f_35)
	df_plan['List_Price5'] = df_plan['List_City'].map(f_5)
	df_plan['List_Price8'] = df_plan['List_City'].map(f_8)

	# Maximum Price City 
	f_maxprice = lambda t: max(t) # Maximum Price

	# Mapping First City
	df_plan['Max_Price35'] = df_plan['List_Price35'].map(f_maxprice)
	df_plan['Max_Price5'] = df_plan['List_Price5'].map(f_maxprice)
	df_plan['Max_Price8'] = df_plan['List_Price8'].map(f_maxprice)
	df_plan['Max_City'] = df_plan.apply(lambda x: f_maxcity(x.List_City, x.List_Price35), axis = 1)

	# Inner City Stop 
	df_plan['Inner_Stops'] = df_plan.apply(lambda x: inner_stops(x.List_City, x.Max_City), axis = 1)
	df_plan['Outer_Stops'] = df_plan.apply(lambda x: outer_stops(x.List_City, x.Max_City), axis = 1)

	# Total Price
	df_plan['Price35'] = df_plan.apply(lambda x: total_price(x.Max_Price35, x.Inner_Stops, x.Outer_Stops, 
		inner_price, outer_price), axis = 1)
	df_plan['Price5'] = df_plan.apply(lambda x: total_price(x.Max_Price5, x.Inner_Stops, x.Outer_Stops, 
		inner_price, outer_price), axis = 1)
	df_plan['Price8'] = df_plan.apply(lambda x: total_price(x.Max_Price8, x.Inner_Stops, x.Outer_Stops, 
		inner_price, outer_price), axis = 1)

	return df_plan